In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_00="/home/mustapha/Downloads/data_65.csv"
DF = pd.read_csv(data_00)
df = DF.iloc[:176]
df1 = DF.iloc[176:].reset_index(drop=True)

df.info()
# df=pd.read_csv(data_00)

## Transforming features to Material Formula ##

In [ ]:
def transform_to_material_formula(row):
    # Extract the values from each column
    a = row['Li stoichiometry']
    b = row['La stoichiometry']
    c = row['Zr stoichiometry']
    x = row['Li site dopant stoichiometry']
    y = row['La site dopant stoichiometry']
    z = row['Zr site dopant stoichiometry']
    M = row['li_dopant']
    N = row['la_dopant']
    K = row['zr_dopant']


    formula = ''


    if a > 0:
        formula += f'Li{a:.1f}'

    # Add M if x > 0
    if x > 0:
        formula += f'{M}{x:.1f}'

    # Add La with its stoichiometry
    if b > 0:
        formula += f'La{b:.1f}'

    # Add N if y > 0
    if y > 0:
        formula += f'{N}{y:.1f}'

    # Add Zr with its stoichiometry
    if c > 0:
        formula += f'Zr{c:.1f}'

    # Add K if z > 0
    if z > 0:
        formula += f'{K}{z:.1f}'
        
    formula += f'O12'
   
    return formula


df['MaterialFormula'] = df.apply(transform_to_material_formula, axis=1)

## Tailored one-hot encoding strategy ##

In [ ]:
def one_hot_encoding_li(df):
    u_l=pd.unique(df["li_dopant"]) # list containing the unique Li site dopants
    d={}
    for x in u_l:        
        if isinstance(x,str): # check if the type of this object is not 'Nan'
            d[x] = [] 
    for i in range(len(df["li_dopant"])):
        for k in d:
            if k == df["li_dopant"][i]:
                d[k].append(df["Li site dopant stoichiometry"][i])
                #if the site of col site is doped with that element
            else:
                d[k].append(0)
                # 0 if not
    return d

In [ ]:
    def one_hot_encoding_la(df):
        u_l=pd.unique(df["la_dopant"]) 
        d={}
        for x in u_l:        
            if isinstance(x,str):
                d[x] = [] 
        for i in range(len(df["la_dopant"])):
            for k in d:
                if k == df["la_dopant"][i]:
                    d[k].append(df["La site dopant stoichiometry"][i])
                    # 1 if the site of col site is doped with that element
                else:
                    d[k].append(0)
                    # 0 if not
        return d

In [ ]:
def one_hot_encoding_zr(df):
    u_l=pd.unique(df["zr_dopant"])
    d={}
    for x in u_l:        
        if isinstance(x,str): 
            d[x] = []
    for i in range(len(df["zr_dopant"])):
        for k in d:
            if k == df["zr_dopant"][i]:
                d[k].append(df["Zr site dopant stoichiometry"][i])
                # 1 if the site of col site is doped with that element
            else:
                d[k].append(0)
                # 0 if not
    return d

In [ ]:
nouveau_dataset={}
li_dopant=pd.DataFrame(one_hot_encoding_li(df)) # replace li_dopant  with one_hot_encoding(li_dopant)
for key in li_dopant.columns:
    nouveau_dataset[key]=list(li_dopant[key]) 
#     nouveau_dataset["li_dopant_"+key]=list(li_dopant[key]) 
li_dopant[19:28]

In [ ]:
la_dopant=pd.DataFrame(one_hot_encoding_la(df))
for key in la_dopant.columns:
    nouveau_dataset[key]=list(la_dopant[key])
#     nouveau_dataset["la_dopant_"+key]=list(la_dopant[key])
la_dopant[19:28]

In [ ]:
zr_dopant=pd.DataFrame(one_hot_encoding_zr(df))
for key in zr_dopant.columns:
    nouveau_dataset[key]=list(zr_dopant[key])
#     nouveau_dataset["zr_dopant_"+key]=list(zr_dopant[key])
zr_dopant[19:28]

In [ ]:
# addition of a new column to define which is the doped site
df.loc[df['Li site dopant stoichiometry'] != 0, 'li'] = 1 # 1 if the material is doped in the Li site
df.loc[df['Li site dopant stoichiometry'] == 0, 'li'] = 0 # 0 if not

df.loc[df['La site dopant stoichiometry'] != 0, 'la'] = 1 # 1 if the material is doped in the Li site
df.loc[df['La site dopant stoichiometry'] == 0, 'la'] = 0 # 0 if not

df.loc[df['Zr site dopant stoichiometry'] != 0, 'zr'] = 1 # 1 if the material is doped in the Li site
df.loc[df['Zr site dopant stoichiometry'] == 0, 'zr'] = 0 # 0 if not

df.head(-1)

In [ ]:
new_dataset=pd.DataFrame(nouveau_dataset) # New dataset 
# new_dataset[19:28]

In [ ]:
print("Shape of df:", df.shape)
print("Shape of nouveau_dataset:", new_dataset.shape)
    
# adding dopant elements to construct  DataFrame X    
X= pd.concat([df, new_dataset], axis=1)

# X.info()
print("Shape of df:", X.shape)

# Y.shape
X.info()

In [ ]:
# Droping columns to perform feature combinations on 'df'

X = X.drop(['Ionic conductivity'],axis=1)
X = X.drop(['Publication year','Quality of ionic conductivity','Doping strategy'], axis=1)
# X = X.drop(['T of the conductivity C'], axis=1)
X = X.drop(['MaterialFormula','source'],axis=1)
X = X.drop(['li','la','zr'], axis=1)
X = X.drop(['Li site dopant ionic radius','La site dopant ionic radius','Zr site dopant ionic radius'], axis=1)
# X = X.drop(['Li stoichiometry','La stoichiometry', 'Zr stoichiometry'], axis=1)
X = X.drop(['Li site dopant stoichiometry','La site dopant stoichiometry','Zr site dopant stoichiometry'], axis=1)
X = X.drop(['Li site dopant electroneg.','La site dopant electroneg.','Zr site dopant electroneg.'], axis=1)
X = X.drop(['Li site dopant ion charge','La site dopant ion charge','Zr site dopant ion charge'], axis=1)
X = X.drop(['Li site dopant crystal rad.','La site dopant crystal rad.','Zr site dopant crystal rad.'], axis=1)
# X = X.drop(['Li site dopant electroneg.','La site dopant electroneg.','Zr site dopant electroneg.'], axis=1)
X = X.drop(['Li site dopant molar mass','La site dopant molar mass','Zr site dopant molar mass'], axis=1)
X = X.drop(['Li site dopant atomic number','La site dopant atomic number','Zr site dopant atomic number'], axis=1)
X = X.drop(['Li site dopant e_ionisation','La site dopant e_ionisation','Zr site dopant e_ionisation'], axis=1)
X = X.drop(['Li site dopant electron affinity','La site dopant electron affinity','Zr site dopant electron affinity'], axis=1)
X = X.drop(['Li site dopant atomic radius','La site dopant atomic radius','Zr site dopant atomic radius'], axis=1)
X = X.drop(['li_dopant','la_dopant','zr_dopant'], axis=1)

X.info()

In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd

##########################

knn_imputer = KNNImputer(n_neighbors=3)
# Fit and transform the data to impute the missing values
X_knn = knn_imputer.fit_transform(X)

# Convert back X_knn back to a DataFrame
X_knn = pd.DataFrame(data=X_knn, columns=X.columns)

Y = X_knn['log_cond']

# Drop 'log_cond' 
X_knn = X_knn.drop('log_cond', axis=1)


##########################

# # Sort the DataFrame by the "log_cond" column
# X_sorted = X.sort_values(by='log_cond')

# # # Perform linear interpolation on the "Relative density" column
# X_filled = X_sorted.interpolate(method='linear', limit_direction='both', inplace=False)
# Y=X_filled['log_cond']

# X_filled=X_filled.drop('log_cond', axis=1)


# X_filled.info()

In [ ]:
X_knn.info()

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# List of regression models 
models = [
    HistGradientBoostingRegressor(),
    LinearRegression(),
    Ridge(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    KNeighborsRegressor(),
    XGBRegressor(),
    LGBMRegressor(),
    CatBoostRegressor()
]

# Create a list to store the results
results = []

# number of iterations
num_iterations = 100

# Iterating over the models and evaluate each one 100 times
for model in models:
    avg_mse = 0
    avg_r2 = 0
    
    model_name = type(model).__name__
    
    for _ in range(num_iterations):
        # Splitting data into training and testing sets for each iteration
        X_train, X_test, Y_train, Y_test = train_test_split(X_knn, Y, test_size=0.25, random_state=42, shuffle=True)
        
        # Model fitting
        model.fit(X_train, Y_train)
        Y_train_pred = model.predict(X_train)
        Y_test_pred = model.predict(X_test)
        
        # Mean squared error & R2-score
        mse = mean_squared_error(Y_test, Y_test_pred)
        r2 = r2_score(Y_test, Y_test_pred)
        
        
        
        # Average metrics update
        avg_mse += mse
        avg_r2 += r2
    
    # Calculating the average metrics over the iterations
    avg_mse /= num_iterations
    avg_r2 /= num_iterations
    results.append({"Model": model_name, "R-squared (R2)": avg_r2, "Mean Squared Error (MSE)": avg_mse})


results_df = pd.DataFrame(results)



In [ ]:
results_df.sort_values(by='R-squared (R2)', ascending=False, inplace=True)
# Reset the indexes to start from 1
results_df.reset_index(drop=True, inplace=True)
results_df

## CatBoostRegressor Model ##

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X_knn, Y, test_size=0.25, random_state=42, shuffle=True)


# model1=CatBoostRegressor(bagging_temperature = 0.5, depth = 3, iterations = 214, l2_leaf_reg = 7,
#                         learning_rate = 0.3, random_strength = 0.5)
model1=CatBoostRegressor()

model1.fit(X_train, Y_train)
Y_train_pred = model1.predict(X_train)
Y_test_pred = model1.predict(X_test)


mse = mean_squared_error(Y_test, Y_test_pred)
r2_cat = r2_score(Y_test, Y_test_pred)
print('R2 score for the CatBoost Model is: ', r2_cat)

In [ ]:
# Plotting Exact vs Predictions
plt.figure(figsize=(8, 8))

# Plot for the training set (red)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.5, label='Train Predictions (CatBoost)')

# Plot for the test set (blue)
plt.scatter(Y_test, Y_test_pred, color='red', alpha=0.5, label='Test Predictions (CatBoost)')

plt.plot( [-8,-2.5], [-8,-2.5], 'k--', lw=2)
plt.title('Actual vs. Predicted Values for CatBoost')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
# Inverse transformation

inv_Y_test_pred=np.exp(Y_test_pred*np.log(10))
inv_Y_test=np.exp(Y_test*np.log(10))

 
inv_r2_test = r2_score(inv_Y_test, inv_Y_test_pred)

r2_cat = r2_score(Y_test, Y_test_pred)

print('Log transformation ')
print('R2 score for the CatBoostRegressor Model is: ', r2_cat)
print('\n')
print('Inverse transformation ')
print(f'R-squared (R2) Score for the log Test Set: {inv_r2_test:.4f} \n')

In [ ]:
error = np.abs(inv_Y_test-inv_Y_test_pred)
err_per = np.abs(inv_Y_test-inv_Y_test_pred)/inv_Y_test
pd.set_option('display.float_format', '{:.10f}'.format)
X_test_original = df.loc[X_test.index, ['MaterialFormula','source']]
df2 = pd.DataFrame({"MaterialFormula": X_test_original['MaterialFormula'],"source": X_test_original['source'],
                    "Y_test_log":Y_test,"Y_test_log_pred":Y_test_pred,"Y_test":inv_Y_test,
                    "Y_test_pred":inv_Y_test_pred,"error": error,"error percentage": err_per})

pd.set_option('display.float_format', None)

# Print the DataFrame
df_sorted_descending = df2.sort_values(by='error percentage')
df_sorted_descending


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import randint

# # Define the parameter grid for hyperparameter tuning
# param_dist = {
#     'iterations': randint(50, 500),
#     'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
#     'depth': randint(1, 10),
#     'l2_leaf_reg': randint(1, 10),
#     'random_strength': [0.1, 0.5, 1, 2, 5],
#     'bagging_temperature': [0.1, 0.5, 1, 2, 5],
# }

# # Initialize the CatBoostRegressor model
# model = CatBoostRegressor()

# # Perform RandomizedSearchCV for hyperparameter tuning
# random_search = RandomizedSearchCV(
#     model,
#     param_distributions=param_dist,
#     n_iter=20,  # Adjust the number of iterations as needed
#     cv=3,  # Adjust the number of cross-validation folds as needed
#     scoring='neg_mean_squared_error',
#     verbose=1,
#     n_jobs=-1,
# )

# # Fit the random search model to the data
# random_search.fit(X_train, Y_train)

# # Get the best parameters from the search
# best_params = random_search.best_params_

# # Use the best model for prediction
# best_model = random_search.best_estimator_
# Y_train_pred = best_model.predict(X_train)
# Y_test_pred = best_model.predict(X_test)

# # Calculate metrics for the best model
# mse = mean_squared_error(Y_test, Y_test_pred)
# r2_cat = r2_score(Y_test, Y_test_pred)

# # Print the results
# print('Best hyperparameters:', best_params)
# print('R2 score for the best CatBoostRegressor model:', r2_cat)



In [ ]:
# Test 1:
# Best hyperparameters: {'bagging_temperature': 5, 'depth': 4, 'iterations': 410, 'l2_leaf_reg': 4, 'learning_rate': 0.2, 'random_strength': 0.5}
# R2 score for the best CatBoostRegressor model: 0.8271191424333637

# Test 2:
# Best hyperparameters: {'bagging_temperature': 2, 'depth': 4, 'iterations': 405, 'l2_leaf_reg': 4, 'learning_rate': 0.2, 'random_strength': 2}
# R2 score for the best CatBoostRegressor model: 0.8297602354808238

# Test 3:
# Best hyperparameters: {'bagging_temperature': 2, 'depth': 5, 'iterations': 283, 'l2_leaf_reg': 5, 'learning_rate': 0.2, 'random_strength': 0.5}
# R2 score for the best CatBoostRegressor model: 0.8327860400451366

# Test 4:
# Best hyperparameters: {'bagging_temperature': 0.1, 'depth': 2, 'iterations': 395, 'l2_leaf_reg': 5, 'learning_rate': 0.3, 'random_strength': 5}
# R2 score for the best CatBoostRegressor model: 0.8121874045282357

# Test 5:
# Best hyperparameters: {'bagging_temperature': 0.5, 'depth': 3, 'iterations': 214, 'l2_leaf_reg': 7, 'learning_rate': 0.3, 'random_strength': 0.5}
# R2 score for the best CatBoostRegressor model: 0.8469166559289

# Test 6:
# Best hyperparameters: {'bagging_temperature': 2, 'depth': 4, 'iterations': 147, 'l2_leaf_reg': 3, 'learning_rate': 0.3, 'random_strength': 1}
# R2 score for the best CatBoostRegressor model: 0.8133142715797372
# R2 Score for ionic conductivity: 0.58

## XGBRegressor

In [ ]:
# XGBRegressor

X_train, X_test, Y_train, Y_test = train_test_split(X_knn, Y, test_size=0.25, random_state=42, shuffle=True)

model2 = XGBRegressor()

model2.fit(X_train, Y_train)
Y_train_pred = model2.predict(X_train)
Y_test_pred = model2.predict(X_test)

# Calculate metrics for this iteration
mse = mean_squared_error(Y_test, Y_test_pred)
r2_cat = r2_score(Y_test, Y_test_pred)
print('R2 score for the XGBRegressor Model is: ', r2_cat)

In [ ]:
plt.figure(figsize=(8, 8))

# Plot for the training set (red)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.5, label='Train Predictions (XGBRegressor)')

# Plot for the test set (blue)
plt.scatter(Y_test, Y_test_pred, color='red', alpha=0.5, label='Test Predictions (XGBRegressor)')

plt.plot( [-8,-2.5], [-8,-2.5], 'k--', lw=2)
plt.title('Actual vs. Predicted Values for XGBRegressor')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()